In [1]:
from datasets import load_dataset
import markovify

dataset = load_dataset("biglam/gutenberg-poetry-corpus")
dataset['train']  = dataset['train'][:50000]

### Versurile poeziei

In [2]:
dataset['train']['line']

['The Song of Hiawatha is based on the legends and stories of',
 'many North American Indian tribes, but especially those of the',
 'Ojibway Indians of northern Michigan, Wisconsin, and Minnesota.',
 'They were collected by Henry Rowe Schoolcraft, the reknowned',
 'Schoolcraft married Jane, O-bah-bahm-wawa-ge-zhe-go-qua (The',
 'fur trader, and O-shau-gus-coday-way-qua (The Woman of the Green',
 'Prairie), who was a daughter of Waub-o-jeeg (The White Fisher),',
 'who was Chief of the Ojibway tribe at La Pointe, Wisconsin.',
 'Jane and her mother are credited with having researched,',
 'authenticated, and compiled much of the material Schoolcraft',
 'included in his Algic Researches (1839) and a revision published',
 'in 1856 as The Myth of Hiawatha.  It was this latter revision',
 'that Longfellow used as the basis for The Song of Hiawatha.',
 'Longfellow began Hiawatha on June 25, 1854, he completed it',
 'soon as the poem was  published its popularity was assured.',
 'However, it als

## PROBLEMA 2a - generare versuri

### Generare de versuri

In [3]:
text = markovify.Text(dataset['train']['line'])
for i in range(10):
    print(text.make_sentence())

To Till and keep, and of his conquest.
By force, and at Midnight return'd
To me transgressour, who for the third part of all my conduct, as their King.
Hide me, where subsist?
With fragrance and with them comes a third of Regal port,
And aloft upon the yellow perch, the Sahwa,
Irrevocable, that his regal town on fire,
Draw after him thus answer'd brief.
Some lead their lives, and nature breeds,
A dewie Cloud, and Pillar of State; deep on his front a snowy star he bore.


In [4]:
strofa = []
for i in range(10):
    strofa.append(text.make_sentence())

for vers in strofa:
    print(vers)

On his great Work, that it was good, and of old now faild
Disturbed not, waiting close the happy harbor meet,
Drifting near to where I met Jack Ellis -- my God! but I'm tired, I'm tired....
Who meet with various face, begins
And breath'd a youthful vigor and autumnal green.
Eev'ning arose in Eden, for the rest.
Through the doorways of the prime, yet such wherein appeared
The angelick guards ascended, mute, and how the pipes shrill out, the mad drums beat!
Go, Michael, of celestial food, divine
Man I deceav'd: that which thee to restore


## PROBLEMA 2b - emotii

In [5]:
from textblob import TextBlob
import pandas as pd

def analyseSentiments(text):
    def getSubjectivity(sentence):
        return TextBlob(sentence).sentiment.subjectivity
    
    def getPolarity(sentence):
        return TextBlob(sentence).sentiment.polarity
    
    text['TextBlob_Subjectivity'] = text['strofa'].apply(getSubjectivity)
    text['TextBlob_Polarity'] = text['strofa'].apply(getPolarity)

    def getAnalysis(score):
        if score < 0:
            return 'Negative'
        elif score == 0:
            return 'Neutral'
        else: 
            return 'Positive'
        
    text['TextBlob_Analysis'] = text['TextBlob_Polarity'].apply(getAnalysis)
    return text

### Emotia textului generat

In [6]:
poezie = {
    'strofa': strofa
}
dataFramePoezie = pd.DataFrame(poezie)
emotii = analyseSentiments(dataFramePoezie)
print(emotii)

                                              strofa  TextBlob_Subjectivity  \
0  On his great Work, that it was good, and of ol...               0.516667   
1  Disturbed not, waiting close the happy harbor ...               1.000000   
2  Drifting near to where I met Jack Ellis -- my ...               0.600000   
3                 Who meet with various face, begins               0.500000   
4  And breath'd a youthful vigor and autumnal green.               0.300000   
5              Eev'ning arose in Eden, for the rest.               0.000000   
6  Through the doorways of the prime, yet such wh...               0.500000   
7  The angelick guards ascended, mute, and how th...               0.800000   
8             Go, Michael, of celestial food, divine               0.000000   
9         Man I deceav'd: that which thee to restore               0.000000   

   TextBlob_Polarity TextBlob_Analysis  
0           0.533333          Positive  
1           0.800000          Positive  
2      

## PROBLEMA 2c - sinonime

In [10]:
import gensim.downloader
from numpy.random import rand

glove_vectors = gensim.downloader.load('glove-wiki-gigaword-300')

def find_synonym(word):
    return glove_vectors.most_similar(word)[0][0]


#generarea poeziei
text = markovify.Text(dataset['train']['line'])

strofa = []
for i in range(10):
    strofa.append(text.make_sentence())

#afisare poezie initiala
print("POEZIA INITIALA\n")
for vers in strofa:
    print(vers)

def inlocuireRandomCuSinonime(strofa):
    strofaNoua = []
    for vers in strofa:
        versNou = ""
        for cuvant in vers.split(' '):
            p = rand()
            if p > 0.5:
                try:
                    cuvant = find_synonym(cuvant)
                except:
                    cuvant = cuvant
            versNou += cuvant + ' '
        strofaNoua.append(versNou)
    return strofaNoua


#afisare poezie dupa modificari
print('\nPOEZIA NOUA\n')
strofeNoi = inlocuireRandomCuSinonime(strofa)
for vers in strofeNoi:
    print(vers) 


POEZIA INITIALA

As he sat there,
And when high noon hast gained, and with him raise
But thou art not lovelier than lilacs,--no,
Against his Maker; no decree of God, I have receav'd, to answer thy desire which tends to know
In Rhodope, where woods and waters wonder at the old men,
How easily could God, if He so merrily and madly
He onward came; far off to fright,
Under thy conduct, and, in me, from their vengeance after fate:
Hath touched my sense, flat seems to me and thrust
And all the band,

POEZIA NOUA

As him sat there, 
And when low a.m. thou gained, both both them raising 
But thou art n't lovelier than lilacs,--no, 
Against he Maker; any decrees the God, I they receav'd, help answer thy wish which tend to think 
In Rhodope, where woods and waters wonder where the old men, 
How readily could God, not He so merrily both madly 
He onwards came; than off to fright, 
Under thy conduct, and, where me, back their revenge after fate: 
Hath touched my sense, flat seems to you both thrus

## PROBLEMA 2e - metrica BLEU

In [7]:
from nltk.translate.bleu_score import sentence_bleu

reference = [[cuvant for cuvant in vers] for vers in dataset['train']['line']]
candidate = [cuvant for vers in strofa for cuvant in vers]
score = sentence_bleu(reference, candidate)
print(score)


0.7598277509990912
